In [1]:
!pip install openpyxl nltk

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_dir = "zh2ko_based_on_nllb-200-distilled-600M"
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from transformers import pipeline

translator = pipeline("translation", model=model, tokenizer=tokenizer,
                      src_lang=tokenizer.src_lang,
                      tgt_lang=tokenizer.tgt_lang,
                      max_length=400)

In [4]:
import pandas as pd

test_data_path = "data/test_data.csv"
df_test = pd.read_csv(test_data_path)

source_texts = df_test[tokenizer.src_lang].to_list()  # Source Text (源文本)
references = df_test[tokenizer.tgt_lang].to_list()  # Reference (参考)

In [5]:
from tqdm.notebook import tqdm

# 进行翻译并提取结果
target_texts = [translator(item)[0] for item in tqdm(source_texts)]
target_texts = [item['translation_text'] for item in target_texts]

  0%|          | 0/24000 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [6]:
# !pip install datasets

In [7]:
# from datasets import Dataset

# # 创建 Dataset 对象
# dataset = Dataset.from_dict({"text": source_texts})

# # 定义翻译函数
# def translate_batch(batch):
#     translations = translator(batch["text"])
#     batch["translation_text"] = [t["translation_text"] for t in translations]
#     return batch

# # 使用 tqdm 显示进度条
# translated_dataset = dataset.map(translate_batch, batched=True, batch_size=100, desc="Translating")
# target_texts = [item["translation_text"] for item in translated_dataset]

In [8]:
df_result = pd.DataFrame({
    "source_texts": source_texts,
    "target_texts": target_texts,
    "references": references,
})
df_result

,source_texts,target_texts,references
0,"当前的法规要求燃油经济性水平可以使汽油车每升行驶15公里左右,但2015年该标准将提高到18...",그러나 자동차를 주행하는 기준은은 15대 정도 정도 높일 수 있다.,현행 규제에서는 휘발유차의 경우 1ℓ당 약 15km를 주행할 수 있는 연비 수준을 ...
1,"起亚汽车公开了以加强电动汽车和未来出行服务为核心的中长期未来战略""Plan S""",기아차차는 미래차 서비스 '플랜'을 강화한다.,기아자동차가 전기자동차와 미래 모빌리티 서비스 강화를 골자로 하는 중장기 미래 전략...
2,"希望进行检查的顾客可以在现代汽车车辆管理应用程序""MyCar Story""或起亚汽车移动顾客...",차량 관리 프로그램인 '모차'K'를 할인해'를 원'을'를'를 원할한다.,점검을 희망하는 고객은 현대차 차량 관리 앱 '마이카 스토리' 또는 기아차 모바일 ...
3,"进口汽车业界正在关注""奥迪的回归""在此前梅赛德斯-奔驰独占鳌头的市场会带来怎样的变化。",이 같은 시장에서 현대차가차가 '모국'의'가'가'가'가'가'가'가(주(주(주)가(주...,수입차업계는 '아우디의 귀환'이 그간 메르세데스-벤츠의 독주 체제였던 시장에 어떤 ...
4,"比起内燃机,正在研究以电动汽车的电池、氢气电动车的氢气罐等环保车为主的对应方案。",내연기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기기...,"내연기관보다 전기차의 배터리, 수소전기차의 수소탱크 등 친환경차 위주로 대응 방안을..."
...,...,...,...
23995,"但由于决定继续与现代汽车进行协商,因此现代汽车收购万都的可能性进一步增大。",그러나 현대차와 협의이해가 계속돼 더 많은 차종의 구매 가능성이 높아졌다.,그러나 현대자동차와는 계속 협상을 벌이기로 해 현대차가 만도를 인수할 가능성이 한층...
23996,"起亚汽车介绍说,该款车型扩大了散热器护栅的大小,打破了前兆灯的界限,进化成了""老虎脸""形态。",자동차는 '대형화화'을 확대했다.,라디에이터 그릴의 크기를 키우고 헤드램프 경계를 허물어 '타이거 페이스'(호랑이 얼...
23997,"另外,日产上月因""新型冠状病毒""关闭了主要市场的汽车展厅。",또 닛산은 '모형'차'로도'로'로'로'로'로'로'로'로'로'로 '모형형'로'로'로'...,또한 닛산은 지난달 코로나19로 인해 주요 시장의 자동차 전시장을 폐쇄했다.
23998,"继之前推出3.0柴油版后,又增加了汽油版2.5涡轮增压发动机和3.5涡轮增压发动机,完成了共...",디젤 모델인 2.5을 추가했다.,"앞서 3.0 디젤 모델을 출시한 데 이어 가솔린 모델의 2.5 터보, 3.5 터보 ..."


In [10]:
file_name = "results_of_{0}".format(model_dir.replace("/", "_"))
df_result.to_excel("{0}.xlsx".format(file_name), index=False)
df_result.to_csv("{0}.csv".format(file_name), index=False)

In [11]:
from nltk.translate.bleu_score import corpus_bleu

# corpus_bleu 的结果更稳健，因为它是基于整个语料库的平均分数计算的，而 sentence_bleu 仅基于单个句子。
# 注：nltk3.8.1和python 3.12有点问题，要么降低python版本，要么按照以下链接的指导去修改bleu_score.py文件
# https://github.com/nltk/nltk/pull/3207
# https://github.com/nltk/nltk/blob/develop/nltk/translate/bleu_score.py
score = corpus_bleu([[reference] for reference in references], target_texts)
score

/root/miniconda3/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/miniconda3/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/miniconda3/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

1.2477935535541866e-231